<a href="https://www.kaggle.com/code/yanelly/querywhiz-ai-powered-data-questions-insights?scriptVersionId=235364428" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🤖 GenAI for Data Analysis: Ask Questions, Get SQL, Understand Results

Welcome! This project demonstrates how to use **Google Gemini AI** to enhance the data analysis process with:
- AI-generated **questions and SQL queries**
- Natural language **summaries of query results**
- Support for **custom CSV uploads** or a built-in **Kaggle dataset**

🎯 Goal: Show how GenAI can act like a smart analyst assistant for any dataset.


# 1. 🔧 Setup: Libraries, API Key, and Imports

In [1]:
# Gemini + API Setup
from google import genai
from google.genai import types
from kaggle_secrets import UserSecretsClient
import warnings

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [2]:
# For API retry handling
from google.api_core import retry

In [3]:
# Displaying Data
import pandas as pd

In [4]:
# SQLite DB
import sqlite3

In [5]:
# Librairies for the inputed csv:
from IPython.display import display
import ipywidgets as widgets
import io

In [6]:
# API Error 
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [7]:
# getting API key
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

# 2. 📂 Load a Dataset (Kaggle OR Upload your own CSV)

## Loading kaggle data

In [8]:
# Data Set: 
data = pd.read_csv('/kaggle/input/amazon-sales-2025/amazon_sales_data 2025.csv')
data

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,ORD0001,14-03-25,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,20-03-25,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,15-02-25,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,19-02-25,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,10-03-25,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending
...,...,...,...,...,...,...,...,...,...,...,...
245,ORD0246,17-03-25,T-Shirt,Clothing,20,2,40,Daniel Harris,Miami,Debit Card,Cancelled
246,ORD0247,30-03-25,Jeans,Clothing,40,1,40,Sophia Miller,Dallas,Debit Card,Cancelled
247,ORD0248,05-03-25,T-Shirt,Clothing,20,2,40,Chris White,Denver,Debit Card,Cancelled
248,ORD0249,08-03-25,Smartwatch,Electronics,150,3,450,Emily Johnson,New York,Debit Card,Cancelled


## Loading input data

> 👉 **Tip**: You can upload your own `.csv` file using the widget below. If you skip this step, the notebook will use a default Amazon Sales dataset.


In [9]:
# Upload CSV widget
upload = widgets.FileUpload(accept='.csv', multiple=False)
display(upload)

# Ask user if they want to upload a CSV
use_upload = input("📤 Do you want to upload your own CSV file? (y/n): ").lower()

if use_upload == 'y':
    # Show the upload widget
    upload = widgets.FileUpload(accept='.csv', multiple=False)
    display(upload)
    print("📤 Please upload your file using the widget above, then run the next cell.")
else:
    print("📂 Skipping upload. We'll use the default dataset instead.")

FileUpload(value=(), accept='.csv', description='Upload')

📤 Do you want to upload your own CSV file? (y/n):  y


FileUpload(value=(), accept='.csv', description='Upload')

📤 Please upload your file using the widget above, then run the next cell.


In [10]:
# Handle upload
def handle_upload():
    if 'upload' in globals() and upload.value:
        for file_info in upload.value:
            content = file_info['content']  # Access the content attribute of the file info
            df = pd.read_csv(io.BytesIO(content))
            df.columns = [col.strip().replace(" ", "_") for col in df.columns]
            return df
    return None  # No file uploaded

df = handle_upload()

if df is None:
    print("⚠️ No user file uploaded — using default Kaggle dataset instead.")
    df = pd.read_csv('/kaggle/input/amazon-sales-2025/amazon_sales_data 2025.csv')
    df.columns = [col.strip().replace(" ", "_") for col in df.columns]

df.head()

⚠️ No user file uploaded — using default Kaggle dataset instead.


,Order_ID,Date,Product,Category,Price,Quantity,Total_Sales,Customer_Name,Customer_Location,Payment_Method,Status
0,ORD0001,14-03-25,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,20-03-25,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,15-02-25,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,19-02-25,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,10-03-25,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending


# 3. 🧠 Ask AI: What Should We Explore?

## prompt for the kaggle data set

In [11]:
# Prompt:
sample = data.head(5).to_markdown()  # Only show a small sample in the prompt
prompt = f"""
Here is a a few rows of our dataset:

{sample}

Based on this dataset, what are some useful questions we should ask during further data analysis?
"""

short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents=prompt
)
print(response.text)

Okay, based on the provided dataset columns, here are some useful questions to ask during further data analysis. I've grouped them by category for clarity:

**Sales & Revenue:**

*   **Overall Sales Performance:**
    *   What is the total revenue generated?
    *   What is the average order value?
    *   What is the distribution of order values?
    *   What are the monthly/quarterly/yearly sales trends? Are sales increasing, decreasing, or stagnant?
*   **Product Performance:**
    *   Which products generate the most revenue?
    *   Which products are most frequently ordered (highest quantity sold)?
    *   What is the average quantity sold per product?
    *   Are there any products that are consistently underperforming?
*   **Category Performance:**
    *   Which product category generates the most revenue?
    *   Which category has the highest average order value?
    *   


## Prompt for the input data

In [12]:
# Prompt:
sample_1 = df.head(5).to_markdown()

prompt_1 = f"""
Here is a few rows of our dataset:

{sample_1}

Based on this dataset, what are some useful questions we should ask during further data analysis?
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents=prompt_1
)

print(response.text)


Okay, based on the provided dataset structure, here are some useful questions to ask during further data analysis, categorized for clarity:

**I. Sales Performance & Trends:**

*   **Overall Sales:**
    *   What is the total sales revenue generated?
    *   What is the average order value?
    *   What is the distribution of order sizes (quantity)?
*   **Temporal Analysis:**
    *   What are the sales trends over time (daily, weekly, monthly)?  (Requires more data over a longer period)
    *   Are there any seasonal patterns in sales? (Requires more data over a longer period)
    *   Are there any specific days or periods with unusually high or low sales?
*   **Product Performance:**
    *   Which product is the best seller?
    *   Which category generates the most revenue?
    *   What is the average quantity sold per product?
    *   Are there


# 4. 🧾 Generate & Run SQL Queries from Natural Language

In [13]:
# description function for both kaggle and user inputed csv:
def describe_table(conn, table_name: str):
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    return [(col[1], col[2]) for col in cursor.fetchall()]

In [14]:
# Query function for both kaggle and user inputed csv: 
def execute_query(conn, sql: str) -> list[list[str]]:
    print(f' - DB CALL: execute_query({sql})')
    cursor = conn.cursor()
    cursor.execute(sql)
    return cursor.fetchall()

## running SQL Queries for the kaggle data set

In [15]:
# Using sqlite3 to create the database
kaggle_conn = sqlite3.connect("sample.db")
data.columns = [col.strip().replace(" ", "_") for col in data.columns]
data.to_sql("data", kaggle_conn, if_exists="replace", index=False)

250

In [16]:
describe_table(kaggle_conn, "data")

[('Order_ID', 'TEXT'),
 ('Date', 'TEXT'),
 ('Product', 'TEXT'),
 ('Category', 'TEXT'),
 ('Price', 'INTEGER'),
 ('Quantity', 'INTEGER'),
 ('Total_Sales', 'INTEGER'),
 ('Customer_Name', 'TEXT'),
 ('Customer_Location', 'TEXT'),
 ('Payment_Method', 'TEXT'),
 ('Status', 'TEXT')]

In [17]:
execute_query(kaggle_conn,"select * from data where Category == 'Footwear'")

 - DB CALL: execute_query(select * from data where Category == 'Footwear')


[('ORD0001',
  '14-03-25',
  'Running Shoes',
  'Footwear',
  60,
  3,
  180,
  'Emma Clark',
  'New York',
  'Debit Card',
  'Cancelled'),
 ('ORD0003',
  '15-02-25',
  'Running Shoes',
  'Footwear',
  60,
  2,
  120,
  'John Doe',
  'Denver',
  'Amazon Pay',
  'Cancelled'),
 ('ORD0004',
  '19-02-25',
  'Running Shoes',
  'Footwear',
  60,
  3,
  180,
  'Olivia Wilson',
  'Dallas',
  'Credit Card',
  'Pending'),
 ('ORD0019',
  '22-03-25',
  'Running Shoes',
  'Footwear',
  60,
  3,
  180,
  'Olivia Wilson',
  'Houston',
  'Credit Card',
  'Completed'),
 ('ORD0046',
  '06-03-25',
  'Running Shoes',
  'Footwear',
  60,
  2,
  120,
  'David Lee',
  'Houston',
  'Debit Card',
  'Cancelled'),
 ('ORD0053',
  '24-03-25',
  'Running Shoes',
  'Footwear',
  60,
  4,
  240,
  'Emily Johnson',
  'Los Angeles',
  'PayPal',
  'Completed'),
 ('ORD0079',
  '09-03-25',
  'Running Shoes',
  'Footwear',
  60,
  2,
  120,
  'Emily Johnson',
  'Denver',
  'Gift Card',
  'Cancelled'),
 ('ORD0080',
  '23-02

## Running SQL Queries from the input data

In [18]:
# 🛢️ Save uploaded data to SQLite
user_conn = sqlite3.connect("sample_1.db")
df.columns = [col.strip().replace(" ", "_") for col in df.columns]
df.to_sql("df", user_conn, if_exists="replace", index=False)

250

In [19]:
describe_table(user_conn,"df")

[('Order_ID', 'TEXT'),
 ('Date', 'TEXT'),
 ('Product', 'TEXT'),
 ('Category', 'TEXT'),
 ('Price', 'INTEGER'),
 ('Quantity', 'INTEGER'),
 ('Total_Sales', 'INTEGER'),
 ('Customer_Name', 'TEXT'),
 ('Customer_Location', 'TEXT'),
 ('Payment_Method', 'TEXT'),
 ('Status', 'TEXT')]

In [20]:
"""Uncomment the line below to execute a query on the user-uploaded dataset."""
# execute_query()

'Uncomment the line below to execute a query on the user-uploaded dataset.'

In [21]:
print("📂 Default dataset loaded into SQLite as 'data'")
print("📂 User-uploaded dataset loaded into SQLite as 'df'")

📂 Default dataset loaded into SQLite as 'data'
📂 User-uploaded dataset loaded into SQLite as 'df'


# Agents

In [22]:
# Agents function for both Kaggle and user-uploaded CSV:
def agent_loop(df, conn, table_name="data"):
    print("🔍 Ask a question about the dataset (or type 'exit'):")

    # Dynamically build schema from df
    schema = f"Table: {table_name}\nColumns:\n"
    for col in df.columns:
        schema += f"- {col}\n"

    # Start interaction loop
    while True:
        user_input = input("\n🧍 You: ")
        if user_input.lower() == "exit":
            break

        # Prompt for SQL
        prompt = f"""
        You are a helpful assistant that answers data questions by generating SQL queries.
        Here is the table schema:
        {schema}
        Question: {user_input}
        Only respond with a valid SQL query.
        """
        
        # Gemini API call using the Kaggle client
        response = client.models.generate_content(  # Make sure `client` is initialized properly
            model='gemini-2.0-flash',
            config=short_config,
            contents=prompt
        )
        sql = response.text.strip().replace("```sql", "").replace("```", "").strip()
        print(f"\n🧾 Cleaned SQL:\n{sql}")

        try:
            result = execute_query(conn, sql)  # Assuming `conn` is a valid connection object
            print("\n📊 Query Results:")
            for row in result:
                print(row)

            # Explanation
            summary_prompt = f"""
            Here is the result of the SQL query:
            {result}
            Explain this result in plain English for a data analyst.
            """
            summary = client.models.generate_content(  # Again, using the Kaggle client here
                model='gemini-2.0-flash',
                config=short_config,
                contents=summary_prompt
            )
            print(f"\n🗣️ Summary:\n{summary.text}")
        except Exception as e:
            print(f"❌ Error: {e}")


## 5. 🤖 Agent Mode: Chat with Your Data W kaggle data and input data

In [23]:
agent_loop(data, kaggle_conn) 

🔍 Ask a question about the dataset (or type 'exit'):



🧍 You:  What are the average sales per order?



🧾 Cleaned SQL:
SELECT AVG(Total_Sales) AS Average_Sales_Per_Order FROM data;
 - DB CALL: execute_query(SELECT AVG(Total_Sales) AS Average_Sales_Per_Order FROM data;)

📊 Query Results:
(975.38,)

🗣️ Summary:
Okay, here's the explanation of the SQL query result `[(975.38,)]` in plain English for a data analyst:

**"The query you ran returned a single row containing a single value, which is 975.38.  The trailing comma and parentheses indicate that this value is part of a tuple (a container holding one or more values), even though it only contains one number in this case."**

**In simpler terms:**

"The SQL query resulted in just one number: 975.38.  The parentheses and comma are just SQL's way of showing that it's technically a single-element list, even though it's just one number."

**Possible Implications (what you might do next):**

As a data analyst, you'll want to consider the context of the query to understand the meaning of this value. Here are some examples:

*   **Average Order 


🧍 You:  exit


In [25]:
"""Uncomment the line below to execute the agent_loop on the user-uploaded dataset."""
# agent_loop(df, user_conn, table_name="df")

🔍 Ask a question about the dataset (or type 'exit'):



🧍 You:  What is the total sales revenue generated?



🧾 Cleaned SQL:
SELECT sum(Total_Sales) FROM df
 - DB CALL: execute_query(SELECT sum(Total_Sales) FROM df)

📊 Query Results:
(243845,)

🗣️ Summary:
Okay, here's the explanation for a data analyst in plain English:

"The query you ran returned a single row with one column. The value in that column is 243845.  Without knowing the query itself, it's impossible to be 100% sure what this means, but the query likely aggregated or counted something, and the result is the number 243845."

**Example scenarios (depending on the query):**

*   **Total number of customers:** "There are 243,845 customers in the customer table."
*   **Total sales revenue (maybe in dollars):** "The total sales revenue is $243,845."
*   **Count of orders:** "There are 243,845 orders in the database."
*   **Number of products in a specific category:** "There are 243,84



🧍 You:  exit
